Step 1.

The first step in the pipeline. First, we have many csv files. Each file contain the mobile traffic from many devices. In this step, we extract mobile traffic from each device into one csv file.

## Import packages

In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Build a function to check whether the file is correct or not

In [ ]:
def show_info(path):
  df = pd.read_csv(path, index_col=0)

  # filter packets that do not contain IP address start with '192.168'
  df0 = df[~df['source_address'].str.contains("192.168.") & ~df['destination_address'].str.contains("192.168.")]
  print('Not 192.168 in both: ', df0.shape[0])

  # filter packets: source IP and des IP start with '192.168'
  df1 = df[df['source_address'].str.contains("192.168.") & df['destination_address'].str.contains("192.168.")]
  print('192.168 in both: ', df1.shape[0])

  # get all IP address of local devices
  # extract in source_address
  temp_df1 = df[df['source_address'].str.contains("192.168.")]
  IP_count1 = temp_df1[['source_address', 'source_port']].groupby('source_address').count()
  IP_count1 = IP_count1.rename(columns={'source_port': 'count'})
  IP_count1 = IP_count1.sort_values(by="count", ascending=False)
  print(IP_count1)

  # extract in destination_address
  temp_df2 = df[df['destination_address'].str.contains("192.168.")]
  IP_count2 = temp_df2[['destination_address', 'destination_port']].groupby('destination_address').count()
  IP_count2 = IP_count2.rename(columns={'destination_port': 'count'})
  IP_count2 = IP_count2.sort_values(by="count", ascending=False)
  print(IP_count2)

## Extract one file into many files

In [ ]:
def extract(path, saved_folder):
  if not (os.path.exists(saved_folder)):
    os.mkdir(saved_folder)

  df = pd.read_csv(path, index_col=0)
  
  # get devices numer
  filename = path.split("/")[-1]
  devices_num = int(filename.split('_')[1])
  print('Number of devices: ', devices_num)
  
  # filter packets that do not contain IP address start with '192.168'
  df0 = df[~df['source_address'].str.contains("192.168.") & ~df['destination_address'].str.contains("192.168.")]
  print('Not contains 192.168.x.x IP address in both source and destination: ', df0.shape[0])

  # filter packets: source IP and des IP start with '192.168'
  df1 = df[df['source_address'].str.contains("192.168.") & df['destination_address'].str.contains("192.168.")]
  print('Contains 192.168.x.x IP address in both source and destination: ', df1.shape[0])

  print('-----------------------------------------------------------------------------')

  # counting the number of IP address in source, then sorting them
  temp_df1 = df[df['source_address'].str.contains("192.168.")]
  IP_count1 = temp_df1[['source_address', 'source_port']].groupby('source_address').count()
  IP_count1 = IP_count1.rename(columns={'source_port': 'count'})
  IP_count1 = IP_count1.sort_values(by="count", ascending=False)
  print(IP_count1)

  # counting the number of IP address in destination, then sorting them
  temp_df2 = df[df['destination_address'].str.contains("192.168.")]
  IP_count2 = temp_df2[['destination_address', 'destination_port']].groupby('destination_address').count()
  IP_count2 = IP_count2.rename(columns={'destination_port': 'count'})
  IP_count2 = IP_count2.sort_values(by="count", ascending=False)
  print(IP_count2)

  print('-----------------------------------------------------------------------------')

  IPs = list(IP_count1.index)[:devices_num]
  for IP in IPs:
    print('Extracting IP: ', IP)
    # filter dataframe by IP address
    sub_df = df[df['source_address'].str.contains(IP) | df['destination_address'].str.contains(IP)].reset_index(drop=True)

    # remove packets of unknown protocol
    sub_df = sub_df[sub_df['protocol'] != 'unknown']

    # save dataframe into csv file
    saved_filename = filename[:-4] + "_" + IP + ".csv"
    saved_path = os.path.join(saved_folder, saved_filename)
    print(saved_path)
    sub_df.to_csv(saved_path)


In [ ]:
app_path = '/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/among_us/raw'
saved_path = '/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/among_us/extracted'

In [ ]:
for filename in os.listdir(app_path):
  print(filename)

amongus_1_4pm_07042021.csv
amongus_1_5pm_07042021_1.csv
amongus_1_2pm_10042021.csv
amongus_1_8pm_07042021_1.csv
amongus_1_9pm_08042021.csv
amongus_1_153m_4pm_06042021.csv
amongus_1_15m_7pm_08032021.csv
amongus_1_20m_2pm_11032021.csv
amongus_1_30m_12pm_09032021.csv
amongus_1_33m_5pm_09032021.csv
amongus_1_37m_3pm_01042021.csv
amongus_1_53m_10pm_08032021.csv
amongus_1_83_5pm_06042021.csv
amongus_1_93_8pm_06042021.csv
amongus_2_30m_7pm_10032021.csv
amongus_1_3pm_08042021.csv
amongus_2_60m_3pm_01042021.csv
amongus_3_47m_3pm_01042021.csv
amongus_2_44m_6pm_08032021.csv
amongus_1_100m_6pm_10032021.csv


In [ ]:
extract(os.path.join(app_path, 'amongus_1_9pm_08042021.csv'), saved_path)

Number of devices:  1
Not contains 192.168.x.x IP address in both source and destination:  0
Contains 192.168.x.x IP address in both source and destination:  473
-----------------------------------------------------------------------------
                 count
source_address        
192.168.137.123  37570
192.168.137.1      784
                     count
destination_address       
192.168.137.123      89005
192.168.137.1          207
192.168.137.255         60
-----------------------------------------------------------------------------
Extracting IP:  192.168.137.123
/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/among_us/extracted/amongus_1_9pm_08042021_192.168.137.123.csv


Extract multiple files

In [ ]:
src_path = '/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data'
apps_li = os.listdir(src_path)

In [ ]:
for app in apps_li:
  app_path = '/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/{}/raw'.format(app)
  saved_path = '/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/{}/extracted'.format(app)

  if not os.path.exists(saved_path):
    for filename in os.listdir(app_path):
      extract(os.path.join(app_path, filename), saved_path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Number of devices:  6
Not contains 192.168.x.x IP address in both source and destination:  2
Contains 192.168.x.x IP address in both source and destination:  1753
-----------------------------------------------------------------------------
                  count
source_address         
192.168.137.16   110658
192.168.137.2    109756
192.168.137.218  106074
192.168.137.163  105016
192.168.137.175   58666
192.168.137.43    52400
192.168.137.93    49560
192.168.137.1      2476
192.168.1.1          21
                      count
destination_address        
192.168.137.163      183879
192.168.137.16       183632
192.168.137.2        183571
192.168.137.218      181447
192.168.137.43       118930
192.168.137.175      118350
192.168.137.93        81724
192.168.137.1           738
192.168.137.255         255
-----------------------------------------------------------------------------
Extracting IP:  192.168.137.16
/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/diijam/extracted/diij

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Number of devices:  6
Not contains 192.168.x.x IP address in both source and destination:  0
Contains 192.168.x.x IP address in both source and destination:  465
-----------------------------------------------------------------------------
                 count
source_address        
192.168.137.209  33576
192.168.137.249  30489
192.168.137.150  19125
192.168.137.125  19005
192.168.137.218  15858
192.168.137.18   15373
192.168.137.1      697
                     count
destination_address       
192.168.137.209      58468
192.168.137.249      58289
192.168.137.150      57678
192.168.137.125      32618
192.168.137.218      28692
192.168.137.18       28127
192.168.137.1          200
192.168.137.255         63
-----------------------------------------------------------------------------
Extracting IP:  192.168.137.209
/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/diijam/extracted/diijam_6_75m_9pm_06032021_192.168.137.209.csv
Extracting IP:  192.168.137.249
/content/drive/My Dri

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


...

Number of devices:  5
Not contains 192.168.x.x IP address in both source and destination:  3
Contains 192.168.x.x IP address in both source and destination:  1603
-----------------------------------------------------------------------------
                  count
source_address         
192.168.137.69   541890
192.168.137.195  528465
192.168.137.132  413082
192.168.137.120  372703
192.168.137.150  228824
192.168.137.83    89071
192.168.137.1       930
192.168.137.112      49
                       count
destination_address         
192.168.137.195      1605992
192.168.137.69       1558764
192.168.137.120       520042
192.168.137.132       414570
192.168.137.150       314861
192.168.137.83        124096
192.168.137.1            769
192.168.137.112           70
192.168.137.255           39
-----------------------------------------------------------------------------
Extracting IP:  192.168.137.69
/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/ola_party/extracted/olaparty_5_1h3

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Number of devices:  8
Not contains 192.168.x.x IP address in both source and destination:  10
Contains 192.168.x.x IP address in both source and destination:  3786
-----------------------------------------------------------------------------
                  count
source_address         
192.168.137.6    789435
192.168.137.231  132102
192.168.137.28   101760
192.168.137.221   79556
192.168.137.192   55999
192.168.137.184   55371
192.168.137.253    6568
192.168.137.12     2884
192.168.137.1      2390
                      count
destination_address        
192.168.137.28       179805
192.168.137.231      131598
192.168.137.221       92705
192.168.137.6         85756
192.168.137.184       55941
192.168.137.192       55122
192.168.137.253        8787
192.168.137.12         2916
192.168.137.1          1791
192.168.137.255          99
-----------------------------------------------------------------------------
Extracting IP:  192.168.137.6
/content/drive/My Drive/AI PROJECTS/GNN/dataset/ra

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Number of devices:  5
Not contains 192.168.x.x IP address in both source and destination:  2
Contains 192.168.x.x IP address in both source and destination:  8128
-----------------------------------------------------------------------------
                   count
source_address          
192.168.137.192  1242369
192.168.137.231   887720
192.168.137.12    882340
192.168.137.221   749277
192.168.137.253   223889
192.168.137.1       4219
                       count
destination_address         
192.168.137.192      1243555
192.168.137.221       961361
192.168.137.231       893700
192.168.137.12        891468
192.168.137.253       384718
192.168.137.1           4021
192.168.137.255           36
-----------------------------------------------------------------------------
Extracting IP:  192.168.137.192
/content/drive/My Drive/AI PROJECTS/GNN/dataset/raw_data/may/extracted/may_5_4h_1pm_12042021_192.168.137.192.csv
Extracting IP:  192.168.137.231
/content/drive/My Drive/AI PROJECTS/GNN/dat